In [1]:
import pandas as pd

# Load the data from the 'data_processed.csv' file 
data = pd.read_csv('data_processed.csv', low_memory=False)

In [2]:
#There was an issue where there were nan values which correspond to a particular file 
#columns_with_nan = data.columns[data.isna().any()].tolist()
#columns_with_nan

In [3]:
#rows_with_nan = data[data.isna().any(axis=1)]
#rows_with_nan

In [4]:
# We have an error ValueError: could not convert string to float: ' '
#non_numeric_columns = data.select_dtypes(exclude=['float64', 'int64']).columns.tolist()
#print("Columns with non-numeric data:", non_numeric_columns)
#for column in non_numeric_columns:
#    print(f"Contents of rows in '{column}' column:")
#    print(data[data[column].notna()][column])
#    print("\n")
    
#column_name = non_numeric_columns[0]
# Get the unique values in the specified column
#unique_values = data[column_name].unique()

# Print the unique values
#print("Unique values in column", column_name, ":")
#for value in unique_values:
#    print(value)

In [5]:
#has_nan_values = data['Number sheathing panels'].isna().any().any()
#has_nan_values

In [6]:
#empty_string_mask = data['Number sheathing panels'] == ' '

# Use the boolean mask to filter the rows with empty strings
#rows_with_empty_strings = data[empty_string_mask]
#rows_with_empty_strings

In [7]:
#data['Number sheathing panels'] = pd.to_numeric(data['Number sheathing panels'], errors='coerce')
#has_nan_values = data['Number sheathing panels'].isna().any().any()
#has_nan_values
#nan_rows = data[data["Number sheathing panels"].isna()]
#print(nan_rows)

In [8]:
data['Number sheathing panels'] = pd.to_numeric(data['Number sheathing panels'], errors='coerce')
has_nan_values = data.isna().any().any()
has_nan_values

False

In [9]:
#Split into X and Y:

# Since the data is alredy splitted into X and Y
# Find the index of the "Story Area" column
story_area_index = data.columns.get_loc("Story Area")

# Split the data into X and Y
X = data.iloc[:, :story_area_index]  # Features from the beginning up to "Story Area"
Y = data.iloc[:, story_area_index:]  # 


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define a list of estimators with their respective hyperparameters
estimators = [
    (LinearRegression(), {'estimator__fit_intercept': [True, False]}),
    (RandomForestRegressor(), {'estimator__n_estimators': [3, 5, 10]})
]

best_model = None
best_mse = float('inf')

# Iterate through the estimators and their hyperparameters
for estimator, param_grid in estimators:
    regressor = MultiOutputRegressor(estimator)
    grid_search = GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, Y_train)
    
    Y_pred = grid_search.predict(X_test)
    mse = mean_squared_error(Y_test, Y_pred)
    
    if mse < best_mse:
        best_mse = mse
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

# Use the best model for prediction and evaluation
Y_pred = best_model.predict(X_test)
best_mse = mean_squared_error(Y_test, Y_pred)

print(f"Best Mean Squared Error: {best_mse}")
print(f"Best Hyperparameters: {best_params}")


Best Mean Squared Error: 0.3478623717331742
Best Hyperparameters: {'estimator__n_estimators': 10}


In [24]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define a list of estimators with their respective hyperparameters
estimators = [
    (LinearRegression(), {'estimator__fit_intercept': [True, False]}),
    (Ridge(), {'estimator__alpha': [0.1, 1.0, 10.0], 'estimator__fit_intercept': [True, False]}),
    (Lasso(), {'estimator__alpha': [0.1, 1.0, 10.0], 'estimator__fit_intercept': [True, False]}),
    (RandomForestRegressor(), {'estimator__n_estimators': [5, 10, 20]})
]


best_model = None
best_mse = float('inf')
results = []  

# Iterate through the estimators and their hyperparameters
for estimator, param_grid in estimators:
    regressor = MultiOutputRegressor(estimator)
    grid_search = GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, Y_train)
    
    Y_pred = grid_search.predict(X_test)
    mse = mean_squared_error(Y_test, Y_pred)
    
    results.append({
        'Estimator': estimator.__class__.__name__,
        'Parameters': grid_search.best_params_,
        'MSE': mse
    })
    
    if mse < best_mse:
        best_mse = mse
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

# Use the best model for prediction and evaluation
Y_pred = best_model.predict(X_test)
best_mse = mean_squared_error(Y_test, Y_pred)

print(f"Best Mean Squared Error: {best_mse}")
print(f"Best Hyperparameters: {best_params}")
print(f"Best Model: {best_model}")


Best Mean Squared Error: 0.3200798384753744
Best Hyperparameters: {'estimator__n_estimators': 20}
Best Model: MultiOutputRegressor(estimator=RandomForestRegressor(n_estimators=20))


In [113]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(49948, 184)
(12488, 184)
(49948, 7)
(12488, 7)


In [114]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Modify the output layer to match the number of target columns (7)
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(7)  # Modify the number of output neurons to 7
])

# Compile the model with appropriate loss for regression
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test))

# Predict on the test set
Y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error: {mse}")



Epoch 1/100
1561/1561 [==============================] - 3s 1ms/step - loss: 3.1778 - val_loss: 1.6409
Epoch 2/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.5771 - val_loss: 1.4848
Epoch 3/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.4560 - val_loss: 1.4027
Epoch 4/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.3733 - val_loss: 1.3540
Epoch 5/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.3202 - val_loss: 1.3120
Epoch 6/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.2819 - val_loss: 1.2857
Epoch 7/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.2481 - val_loss: 1.2704
Epoch 8/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.2203 - val_loss: 1.2730
Epoch 9/100
1561/1561 [==============================] - 2s 1ms/step - loss: 1.1933 - val_loss: 1.2336
Epoch 10/100
1561/1561 [==============================] - 2s 1ms/step - l

Epoch 80/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8795 - val_loss: 1.2114
Epoch 81/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8767 - val_loss: 1.1888
Epoch 82/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8739 - val_loss: 1.1986
Epoch 83/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8679 - val_loss: 1.2005
Epoch 84/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8686 - val_loss: 1.2107
Epoch 85/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8657 - val_loss: 1.1974
Epoch 86/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8595 - val_loss: 1.2206
Epoch 87/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8621 - val_loss: 1.2034
Epoch 88/100
1561/1561 [==============================] - 2s 1ms/step - loss: 0.8591 - val_loss: 1.2022
Epoch 89/100
1561/1561 [==============================] - 2s 1ms

Mean Squared Error: 1.1954145740483273

In [117]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the CNN model
model = keras.Sequential([
    layers.Reshape((184, 1), input_shape=(184,)),  # Reshape input data
    layers.Conv1D(32, 3, activation='relu'),  # Conv1D for 1D data
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    
    # Fully connected layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='linear')  # Output layer for regression with 7 units
])

# Compile the model with appropriate loss for regression
model.compile(optimizer='adam',
              loss='mean_squared_error',  # MSE loss for regression
              metrics=['mean_absolute_error'])  # Optional: You can track other metrics like MAE

# Print the model summary
model.summary()


# Compile the model with appropriate loss for regression
model.compile(optimizer='adam',
              loss='mean_squared_error',  # MSE loss for regression
              metrics=['mean_absolute_error'])  # Optional: You can track other metrics like MAE

# Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test))

# Predict on the test set
Y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error: {mse}")

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 184, 1)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 182, 32)           128       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 91, 32)           0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 89, 64)            6208      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 44, 64)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 42, 128)          

1561/1561 [==============================] - 21s 13ms/step - loss: 1.0057 - mean_absolute_error: 0.3807 - val_loss: 1.1026 - val_mean_absolute_error: 0.3940
Epoch 39/50
1561/1561 [==============================] - 19s 12ms/step - loss: 1.0031 - mean_absolute_error: 0.3794 - val_loss: 1.1215 - val_mean_absolute_error: 0.3947
Epoch 40/50
1561/1561 [==============================] - 17s 11ms/step - loss: 1.0011 - mean_absolute_error: 0.3800 - val_loss: 1.1115 - val_mean_absolute_error: 0.3931
Epoch 41/50
1561/1561 [==============================] - 18s 11ms/step - loss: 0.9985 - mean_absolute_error: 0.3780 - val_loss: 1.0982 - val_mean_absolute_error: 0.3903
Epoch 42/50
1561/1561 [==============================] - 19s 12ms/step - loss: 0.9975 - mean_absolute_error: 0.3784 - val_loss: 1.1125 - val_mean_absolute_error: 0.4016
Epoch 43/50
1561/1561 [==============================] - 19s 12ms/step - loss: 0.9936 - mean_absolute_error: 0.3757 - val_loss: 1.1049 - val_mean_absolute_error: 0.385

Mean Squared Error: 1.1136010892074086

Mean Squared Error: 1.1744087515632793


In [134]:
num_columns_to_round = 5
rounded_Y_pred = Y_pred.copy()  # Make a copy to avoid modifying the original array
rounded_Y_pred[:, :num_columns_to_round] = np.round(rounded_Y_pred[:, :num_columns_to_round])
# Calculate Mean Squared Error
mse = mean_squared_error(Y_test, rounded_Y_pred)
print(f"Mean Squared Error: {mse}")

df2 = pd.DataFrame(rounded_Y_pred)

prepared_file_path = 'comparison_rounded.csv'
df2.to_csv(prepared_file_path, index=False)

Mean Squared Error: 1.1380968504715248


In [130]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Create a DataFrame to compare Actual vs. Predicted values
#comparison_df = pd.DataFrame({'Actual': Y_test.flatten(), 'Predicted': Y_pred.flatten()})

# Display the DataFrame
#print(comparison_df)
#print(f"Mean Squared Error (MSE): {mse}")

df = pd.DataFrame(Y_pred)

prepared_file_path = 'comparison.csv'
df.to_csv(prepared_file_path, index=False)

Mean Squared Error (MSE): 1.1136010892074086


For predicting various details about structural walls, including both numerical and categorical information, a good starting point would be to use a neural network architecture that can handle mixed data types. Here, I'll suggest using a feedforward neural network (also known as a multi-layer perceptron or MLP) with multiple output heads, one for each type of prediction.